In [1]:
import numpy as np
from environment.gobang import WrapperGoBang
from agent import Agent
import random
import os
from loguru import logger

In [2]:
class GoBangRandomAgent(Agent):
    def step(self, obs, explore=True, legal_actions=None):
        return {'action': random.choice(legal_actions)}

In [3]:
def do_exp(agent, gobang_n, epochs, test_n, test_freq, result_path):
    random_agent = GoBangRandomAgent(None)
    loss_episode = []
    reward_episode = []
    win_ratio = []
    best_win_ratio = -np.inf
    env = WrapperGoBang(gobang_n)
    for i in range(epochs):
        loss_list, reward_sum = agent.train(env)
        logger.info(f'Episode {i}/{epochs}, average loss: {np.mean(loss_list) if loss_list else -1}, reward sum: {reward_sum}')
        if loss_list:
            loss_episode.append(np.array([i, np.mean(loss_list)]))
            reward_episode.append(np.array([i, reward_sum]))
        
        if i % test_freq == 0:
            win = 0
            for _ in range(test_n):
                win += agent.test(env, random_agent, display=False)
            logger.info(f'Episode {i}/{epochs}, win {win}/{test_n}')
            ratio = 1.0 * win / test_n
            win_ratio.append(np.array([i, ratio]))
            if ratio > best_win_ratio:
                agent.save()
    np.save(os.path.join(result_path, 'loss.npy'), np.array(loss_episode))
    np.save(os.path.join(result_path, 'reward.npy'), np.array(reward_episode))
    np.save(os.path.join(result_path, 'ratio.npy'), np.array(win_ratio))    
    

In [4]:
from agent import DQNAgent
from agent.model import ConvQModel
from agent.configuration import DQNConfig
from agent.exploration import LinearExploration
from agent.memory import PriorityMemory

In [5]:
eval_model = ConvQModel(
    in_channels=2,
    in_h=7,
    in_w=7,
    hidden_channels=32,
    output_dim=49,
    dueling=False
)
target_model = ConvQModel(
    in_channels=2,
    in_h=7,
    in_w=7,
    hidden_channels=32,
    output_dim=49,
    dueling=False
)
exploration = LinearExploration(
    init_epsilon=1.0,
    min_epsilon=0.1,
    epsilon_decay=0.995
)
config = DQNConfig(
    eval_model=eval_model,
    target_model=target_model,
    ckpt_path='./ddqn_p_gobang_n7.ckpt',
    batch_size=32,
    lr=0.001,
    gamma=0.99,
    target_replace_frequency=100,
    capacity=5000,
    max_grad_norm=1.0,
    exploration=exploration,
    ddqn=True,
    memory=PriorityMemory(capacity=5000)
)
agent = DQNAgent(config)
do_exp(agent, 7, 5000, 50, 50, './experiment/ddqn_p_gobang_n7')

2023-05-05 04:45:46.467 | INFO     | __main__:do_exp:10 - Episode 0/5000, average loss: 0.00957626002635691, reward sum: -37
2023-05-05 04:45:54.325 | INFO     | __main__:do_exp:19 - Episode 0/5000, win 24/50
2023-05-05 04:45:54.962 | INFO     | __main__:do_exp:10 - Episode 1/5000, average loss: 0.0031267152091037746, reward sum: -28
2023-05-05 04:45:55.813 | INFO     | __main__:do_exp:10 - Episode 2/5000, average loss: 0.0013460405999446885, reward sum: -39
2023-05-05 04:45:58.023 | INFO     | __main__:do_exp:10 - Episode 3/5000, average loss: 0.0021649768739346814, reward sum: -31
2023-05-05 04:46:01.576 | INFO     | __main__:do_exp:10 - Episode 4/5000, average loss: 0.0022828034318873283, reward sum: -41
2023-05-05 04:46:04.670 | INFO     | __main__:do_exp:10 - Episode 5/5000, average loss: 0.0005041179260330275, reward sum: -36
2023-05-05 04:46:08.021 | INFO     | __main__:do_exp:10 - Episode 6/5000, average loss: 0.001777372465402674, reward sum: -44
2023-05-05 04:46:10.670 | INFO

In [6]:
eval_model = ConvQModel(
    in_channels=2,
    in_h=11,
    in_w=11,
    hidden_channels=32,
    output_dim=121,
    dueling=False
)
target_model = ConvQModel(
    in_channels=2,
    in_h=11,
    in_w=11,
    hidden_channels=32,
    output_dim=121,
    dueling=False
)
exploration = LinearExploration(
    init_epsilon=1.0,
    min_epsilon=0.1,
    epsilon_decay=0.995
)
config = DQNConfig(
    eval_model=eval_model,
    target_model=target_model,
    ckpt_path='./ddqn_p_gobang_n11.ckpt',
    batch_size=32,
    lr=0.001,
    gamma=0.99,
    target_replace_frequency=100,
    capacity=5000,
    max_grad_norm=1.0,
    exploration=exploration,
    ddqn=True
)
agent = DQNAgent(config)
do_exp(agent, 11, 5000, 50, 50, './experiment/ddqn_p_gobang_n11')

2023-05-05 05:16:23.644 | INFO     | __main__:do_exp:10 - Episode 0/5000, average loss: 0.08144160619294427, reward sum: -76
2023-05-05 05:17:14.589 | INFO     | __main__:do_exp:19 - Episode 0/5000, win 20/50
2023-05-05 05:17:16.848 | INFO     | __main__:do_exp:10 - Episode 1/5000, average loss: 0.045814032045625154, reward sum: -76
2023-05-05 05:17:18.563 | INFO     | __main__:do_exp:10 - Episode 2/5000, average loss: 0.032476625036141044, reward sum: -49
2023-05-05 05:17:21.261 | INFO     | __main__:do_exp:10 - Episode 3/5000, average loss: 0.04751465136150581, reward sum: -67
2023-05-05 05:17:23.056 | INFO     | __main__:do_exp:10 - Episode 4/5000, average loss: 0.02218017646370583, reward sum: -47
2023-05-05 05:17:25.938 | INFO     | __main__:do_exp:10 - Episode 5/5000, average loss: 0.06560539377017584, reward sum: -60
2023-05-05 05:17:28.838 | INFO     | __main__:do_exp:10 - Episode 6/5000, average loss: 0.043221528746030656, reward sum: -64
2023-05-05 05:17:31.579 | INFO     | _